In [1]:
#Establish required libraries

import sys
import types
import pandas as pd 
from botocore.client import Config
import ibm_boto3


import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index
from IPython.display import Image

def __iter__(self): return 0

import pandas as pd
import numpy as np 

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


In [2]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385636 entries, 0 to 385635
Data columns (total 72 columns):
Demo_Key                                    385636 non-null int64
Promo_ID                                    385636 non-null object
Promo_CPG_Name                              281228 non-null object
Promo_Name                                  385636 non-null object
Retailer_Promo_Contract_ID                  0 non-null float64
Retailer_Name                               385636 non-null object
Retailer_Store_ID                           385636 non-null object
Product_Unit_UPC                            385636 non-null object
Product_Group_ID                            385635 non-null object
Product_Brand                               385636 non-null object
Product_Brand_Category_Desc                 385636 non-null object
Product_Category                            385636 non-null object
Product_Sub_Category                        385636 non-null object
Product_Type           

In [3]:
#select the fields you need
df_price_elasticity = df_data_1[['Promo_ID','Promo_Name','Retailer_Name','Retailer_Store_ID','Product_Category',
                                 'Product_Sub_Category',
                             'Product_Unit_UPC','Product_Group_ID','Product_Brand','Product_Brand_Category_Desc',
                              'Product_Unit_Description','Retailer_Product_Base_Day_Unit_Volume',
                              'TPP_Unit_Sale_Price','Retailer_Product_Regular_Unit_Price','TPP_Incremental_Cases',
                             'TPP_Baseline_Cases_Volume','TPP_Type','TPP_Start','TPP_End','Product_Units_Per_Case','Product_CPG_COGS_Price']]






In [4]:
#Visually inspect the different products in the table
df_price_elasticity.Product_Sub_Category.value_counts(dropna = False).sort_index()

Cereal         67360
Cleaners       30312
Ice Cream      47152
Trail Mix      74096
Vegetables    116196
Yogurt         50520
Name: Product_Sub_Category, dtype: int64

In [5]:
df_price_elasticity.shape

(385636, 21)

In [6]:
#De-Dup the file
df_price_elasticity=df_price_elasticity.drop_duplicates(subset=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC'], keep='first', inplace=False)
df_price_elasticity.shape




(385636, 21)

In [7]:
#remove all products but the one we us

df_price_elasticity=df_price_elasticity[df_price_elasticity['Product_Sub_Category'] =='Trail Mix']
df_price_elasticity.shape

(74096, 21)

In [8]:
#create a new field for size
df_price_elasticity['SIZE']=pd.DataFrame(df_price_elasticity['Product_Group_ID'].str[-2:])


In [9]:
df_price_elasticity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74096 entries, 281228 to 355323
Data columns (total 22 columns):
Promo_ID                                 74096 non-null object
Promo_Name                               74096 non-null object
Retailer_Name                            74096 non-null object
Retailer_Store_ID                        74096 non-null object
Product_Category                         74096 non-null object
Product_Sub_Category                     74096 non-null object
Product_Unit_UPC                         74096 non-null object
Product_Group_ID                         74096 non-null object
Product_Brand                            74096 non-null object
Product_Brand_Category_Desc              74096 non-null object
Product_Unit_Description                 74096 non-null object
Retailer_Product_Base_Day_Unit_Volume    74096 non-null float64
TPP_Unit_Sale_Price                      74096 non-null float64
Retailer_Product_Regular_Unit_Price      74096 non-null float64


In [10]:
#create a copy of the original data set
sale=df_price_elasticity

In [11]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74096 entries, 281228 to 355323
Data columns (total 22 columns):
Promo_ID                                 74096 non-null object
Promo_Name                               74096 non-null object
Retailer_Name                            74096 non-null object
Retailer_Store_ID                        74096 non-null object
Product_Category                         74096 non-null object
Product_Sub_Category                     74096 non-null object
Product_Unit_UPC                         74096 non-null object
Product_Group_ID                         74096 non-null object
Product_Brand                            74096 non-null object
Product_Brand_Category_Desc              74096 non-null object
Product_Unit_Description                 74096 non-null object
Retailer_Product_Base_Day_Unit_Volume    74096 non-null float64
TPP_Unit_Sale_Price                      74096 non-null float64
Retailer_Product_Regular_Unit_Price      74096 non-null float64


In [12]:
# create rename the sale price to price, create the cases and units field, drop fields you don't need anymore
#label all of the records as sale records

sale=sale.rename(index=str, columns={"TPP_Unit_Sale_Price": "PRICE"})
sale['CASES'] = sale['TPP_Baseline_Cases_Volume'] + sale['TPP_Incremental_Cases'] 
sale['GROUP']='ONSALE'
sale['UNITS']=sale['CASES']*sale['Product_Units_Per_Case']
sale=sale.drop(['TPP_Baseline_Cases_Volume', 'TPP_Incremental_Cases','Retailer_Product_Regular_Unit_Price','Retailer_Product_Base_Day_Unit_Volume'], axis=1)

In [13]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74096 entries, 281228 to 355323
Data columns (total 21 columns):
Promo_ID                       74096 non-null object
Promo_Name                     74096 non-null object
Retailer_Name                  74096 non-null object
Retailer_Store_ID              74096 non-null object
Product_Category               74096 non-null object
Product_Sub_Category           74096 non-null object
Product_Unit_UPC               74096 non-null object
Product_Group_ID               74096 non-null object
Product_Brand                  74096 non-null object
Product_Brand_Category_Desc    74096 non-null object
Product_Unit_Description       74096 non-null object
PRICE                          74096 non-null float64
TPP_Type                       74096 non-null object
TPP_Start                      74096 non-null object
TPP_End                        74096 non-null object
Product_Units_Per_Case         74096 non-null float64
Product_CPG_COGS_Price         74096 non

In [14]:
#copy the orginal data set, create the non-sale prices
nosale=df_price_elasticity


In [15]:
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74096 entries, 281228 to 355323
Data columns (total 22 columns):
Promo_ID                                 74096 non-null object
Promo_Name                               74096 non-null object
Retailer_Name                            74096 non-null object
Retailer_Store_ID                        74096 non-null object
Product_Category                         74096 non-null object
Product_Sub_Category                     74096 non-null object
Product_Unit_UPC                         74096 non-null object
Product_Group_ID                         74096 non-null object
Product_Brand                            74096 non-null object
Product_Brand_Category_Desc              74096 non-null object
Product_Unit_Description                 74096 non-null object
Retailer_Product_Base_Day_Unit_Volume    74096 non-null float64
TPP_Unit_Sale_Price                      74096 non-null float64
Retailer_Product_Regular_Unit_Price      74096 non-null float64


In [16]:
#create the cases and price field
nosale=nosale.rename(index=str, columns={"TPP_Baseline_Cases_Volume": "CASES", "Retailer_Product_Regular_Unit_Price":"PRICE"})


In [17]:
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74096 entries, 281228 to 355323
Data columns (total 22 columns):
Promo_ID                                 74096 non-null object
Promo_Name                               74096 non-null object
Retailer_Name                            74096 non-null object
Retailer_Store_ID                        74096 non-null object
Product_Category                         74096 non-null object
Product_Sub_Category                     74096 non-null object
Product_Unit_UPC                         74096 non-null object
Product_Group_ID                         74096 non-null object
Product_Brand                            74096 non-null object
Product_Brand_Category_Desc              74096 non-null object
Product_Unit_Description                 74096 non-null object
Retailer_Product_Base_Day_Unit_Volume    74096 non-null float64
TPP_Unit_Sale_Price                      74096 non-null float64
PRICE                                    74096 non-null float64
TPP_I

In [18]:
#Calculate the number of Units
nosale['UNITS']=nosale['CASES']*nosale['Product_Units_Per_Case']

In [19]:
#identify the prices as non-sale prices.
nosale['GROUP']='NOTONSALE'


#Remove stuff you don't need


nosale=nosale.drop([ 'TPP_Incremental_Cases','TPP_Unit_Sale_Price','Retailer_Product_Base_Day_Unit_Volume','TPP_Type'], axis=1)
nosale['TPP_Type']='N'

In [20]:
nosale.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Category,Product_Sub_Category,Product_Unit_UPC,Product_Group_ID,Product_Brand,Product_Brand_Category_Desc,...,PRICE,CASES,TPP_Start,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,UNITS,GROUP,TPP_Type
281228,Brandega-17919,Back to School,Botann's,12345-94249-107,Snack Foods,Trail Mix,44309-79016,MM-SF-TM-B-16,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,...,8.81,1.0,2019-08-26,2019-09-04,25.0,6.19,16,25.0,NOTONSALE,N
281229,Brandega-17919,Back to School,Botann's,12345-94249-107,Snack Foods,Trail Mix,44309-79116,MM-SF-TM-B-16,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,...,8.81,1.0,2019-08-26,2019-09-04,25.0,6.19,16,25.0,NOTONSALE,N
281230,Brandega-17919,Back to School,Botann's,12345-94249-107,Snack Foods,Trail Mix,44309-79216,MM-SF-TM-S-16,Mitan Munchies,Mitan Munchies Trail Mix Spicy 16 oz,...,8.81,1.0,2019-08-26,2019-09-04,25.0,6.19,16,25.0,NOTONSALE,N
281231,Brandega-17919,Back to School,Botann's,12345-94249-107,Snack Foods,Trail Mix,44309-79316,MM-SF-TM-H-16,Mitan Munchies,Mitan Munchies Trail Mix Healthy 16 oz,...,8.81,1.0,2019-08-26,2019-09-04,25.0,6.19,16,25.0,NOTONSALE,N
281232,Brandega-17919,Back to School,Botann's,12345-94249-107,Snack Foods,Trail Mix,44309-79416,MM-SF-TM-B-16,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,...,8.81,1.0,2019-08-26,2019-09-04,25.0,6.19,16,25.0,NOTONSALE,N


In [21]:
sale.info()
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74096 entries, 281228 to 355323
Data columns (total 21 columns):
Promo_ID                       74096 non-null object
Promo_Name                     74096 non-null object
Retailer_Name                  74096 non-null object
Retailer_Store_ID              74096 non-null object
Product_Category               74096 non-null object
Product_Sub_Category           74096 non-null object
Product_Unit_UPC               74096 non-null object
Product_Group_ID               74096 non-null object
Product_Brand                  74096 non-null object
Product_Brand_Category_Desc    74096 non-null object
Product_Unit_Description       74096 non-null object
PRICE                          74096 non-null float64
TPP_Type                       74096 non-null object
TPP_Start                      74096 non-null object
TPP_End                        74096 non-null object
Product_Units_Per_Case         74096 non-null float64
Product_CPG_COGS_Price         74096 non

In [22]:
#include needed fields
nosale = nosale[['Promo_ID',
 'Promo_Name',
 'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS']]

In [23]:
#include needed fields
sale = sale[['Promo_ID',
 'Promo_Name',
 'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS']]

In [24]:
sale.info()
nosale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74096 entries, 281228 to 355323
Data columns (total 19 columns):
Promo_ID                       74096 non-null object
Promo_Name                     74096 non-null object
Retailer_Name                  74096 non-null object
Retailer_Store_ID              74096 non-null object
Product_Group_ID               74096 non-null object
Product_Unit_UPC               74096 non-null object
Product_Brand                  74096 non-null object
Product_Brand_Category_Desc    74096 non-null object
Product_Unit_Description       74096 non-null object
PRICE                          74096 non-null float64
TPP_Type                       74096 non-null object
TPP_Start                      74096 non-null object
TPP_End                        74096 non-null object
Product_Units_Per_Case         74096 non-null float64
Product_CPG_COGS_Price         74096 non-null float64
SIZE                           74096 non-null object
CASES                          74096 no

In [25]:
#append the sale and non-sale records into one dataframe
df_total=pd.DataFrame(sale.append(nosale, ignore_index=True, verify_integrity=False))

In [26]:
#convert the tpp_start which is character to a pandas date field
df_total['date'] = pd.to_datetime(df_total['TPP_Start'])
df_total.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,TPP_Type,TPP_Start,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,TPR,2019-08-26,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79116,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Oriental Mix 16 oz,7.84,TPR,2019-08-26,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26
2,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-S-16,44309-79216,Mitan Munchies,Mitan Munchies Trail Mix Spicy 16 oz,Mitan Munchies Southern Heat 16 oz,7.84,TPR,2019-08-26,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26
3,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-H-16,44309-79316,Mitan Munchies,Mitan Munchies Trail Mix Healthy 16 oz,Mitan Munchies Healthy Trail Mix 16 oz,7.84,TPR,2019-08-26,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26
4,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79416,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Bourbon Street Mix 16 oz,7.84,TPR,2019-08-26,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26


In [27]:
#create a field for year and month
df_total['year'], df_total['month'] = df_total['date'].dt.year, df_total['date'].dt.month
df_total.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,TPP_End,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79116,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Oriental Mix 16 oz,7.84,...,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8
2,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-S-16,44309-79216,Mitan Munchies,Mitan Munchies Trail Mix Spicy 16 oz,Mitan Munchies Southern Heat 16 oz,7.84,...,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8
3,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-H-16,44309-79316,Mitan Munchies,Mitan Munchies Trail Mix Healthy 16 oz,Mitan Munchies Healthy Trail Mix 16 oz,7.84,...,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8
4,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79416,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Bourbon Street Mix 16 oz,7.84,...,2019-09-04,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8


In [28]:
#assign random numbers to each retailer
wookie = pd.DataFrame(df_total.groupby(['Retailer_Store_ID'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)


In [29]:
wookie['Retailer_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()


,Retailer_Store_ID,Retailer_R
0,12345-10024-126,-0.007
1,12345-14213-136,0.002
2,12345-14601-157,-0.022
3,12345-15225-129,0.053
4,12345-15225-167,-0.008


In [30]:
df_total1 =df_total.merge(wookie, on=['Retailer_Store_ID'], how='inner')
df_total1.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Product_Units_Per_Case,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79116,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Oriental Mix 16 oz,7.84,...,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04
2,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-S-16,44309-79216,Mitan Munchies,Mitan Munchies Trail Mix Spicy 16 oz,Mitan Munchies Southern Heat 16 oz,7.84,...,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04
3,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-H-16,44309-79316,Mitan Munchies,Mitan Munchies Trail Mix Healthy 16 oz,Mitan Munchies Healthy Trail Mix 16 oz,7.84,...,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04
4,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79416,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Bourbon Street Mix 16 oz,7.84,...,25.0,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04


In [31]:
#assign random numbers to each product group id
df_total1.Product_Group_ID.value_counts(dropna = False).sort_index()

MM-SF-TM-B-16    37048
MM-SF-TM-B-8     37048
MM-SF-TM-F-16    10104
MM-SF-TM-F-8     10104
MM-SF-TM-H-16    20208
MM-SF-TM-H-8     20208
MM-SF-TM-S-16     6736
MM-SF-TM-S-8      6736
Name: Product_Group_ID, dtype: int64

In [32]:
 wookie = pd.DataFrame(df_total1.groupby(['Product_Group_ID'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

In [33]:
wookie['PRODUCT_GROUP_ID_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()

,Product_Group_ID,PRODUCT_GROUP_ID_R
0,MM-SF-TM-B-16,-0.016
1,MM-SF-TM-B-8,-0.002
2,MM-SF-TM-F-16,-0.004
3,MM-SF-TM-F-8,-0.013
4,MM-SF-TM-H-16,0.013


In [34]:
#!pip install plotly --upgrade

In [35]:
df_total2 =df_total1.merge(wookie, on=['Product_Group_ID'], how='inner')
df_total2.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Product_CPG_COGS_Price,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04,-0.016
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79116,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Oriental Mix 16 oz,7.84,...,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04,-0.016
2,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79416,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Bourbon Street Mix 16 oz,7.84,...,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04,-0.016
3,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79616,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chocolate Monkey Mix 16 oz,7.84,...,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04,-0.016
4,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79716,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Yogurt Trail Mix 16 oz,7.84,...,6.19,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04,-0.016


In [36]:
#assign random numbers to each product unit upc value
df_total2.Product_Unit_UPC.value_counts(dropna = False).sort_index()

44309-79016    3368
44309-79116    3368
44309-79216    3368
44309-79316    3368
44309-79416    3368
44309-79516    3368
44309-79616    3368
44309-79716    3368
44309-79816    3368
44309-79916    3368
44309-80016    3368
44309-80116    3368
44309-80216    3368
44309-80316    3368
44309-80416    3368
44309-80516    3368
44309-80616    3368
44309-80716    3368
44309-80816    3368
44309-80916    3368
44309-81016    3368
44309-81216    3368
44309-81408    3368
44309-81508    3368
44309-81608    3368
44309-81708    3368
44309-81808    3368
44309-81908    3368
44309-82008    3368
44309-82108    3368
44309-82208    3368
44309-82308    3368
44309-82408    3368
44309-82508    3368
44309-82608    3368
44309-82708    3368
44309-82808    3368
44309-82908    3368
44309-83008    3368
44309-83108    3368
44309-83208    3368
44309-83308    3368
44309-83408    3368
44309-83608    3368
Name: Product_Unit_UPC, dtype: int64

In [37]:
wookie = pd.DataFrame(df_total2.groupby(['Product_Unit_UPC'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

wookie['Product_Unit_UPC_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()
df_total3 =df_total2.merge(wookie, on=['Product_Unit_UPC'], how='inner')
df_total3.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,SIZE,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,16,2.0,ONSALE,50.0,2019-08-26,2019,8,0.04,-0.016,0.012
1,Brandega-21621,Movie Munchies,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.19,...,16,2.0,ONSALE,50.0,2019-04-04,2019,4,0.04,-0.016,0.012
2,Brandega-69484,Fun Food,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.19,...,16,2.0,ONSALE,50.0,2019-05-04,2019,5,0.04,-0.016,0.012
3,Brandega-72755,Stocking Stuffer,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.19,...,16,2.0,ONSALE,50.0,2019-12-04,2019,12,0.04,-0.016,0.012
4,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,16,1.0,NOTONSALE,25.0,2019-08-26,2019,8,0.04,-0.016,0.012


In [38]:
#assign random numbers to each promo
df_total2.Promo_Name.value_counts(dropna = False).sort_index()

Back to School      37048
Fun Food            37048
Movie Munchies      37048
Stocking Stuffer    37048
Name: Promo_Name, dtype: int64

In [39]:
wookie = pd.DataFrame(df_total3.groupby(['Promo_Name'])['PRICE'].mean())
wookie=wookie.drop(['PRICE'], axis=1)

wookie['Promo_Name_R'] = (np.random.randint(-30, 60, wookie.shape[0]))/1000
wookie.reset_index(level=0, inplace=True)
wookie.head()
df_total4 =df_total3.merge(wookie, on=['Promo_Name'], how='inner')
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,CASES,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2.0,ONSALE,50.0,2019-08-26,2019,8,0.040,-0.016,0.012,0.025
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,1.0,NOTONSALE,25.0,2019-08-26,2019,8,0.040,-0.016,0.012,0.025
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2.0,ONSALE,50.0,2019-08-26,2019,8,0.005,-0.016,0.012,0.025
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,1.0,NOTONSALE,25.0,2019-08-26,2019,8,0.005,-0.016,0.012,0.025
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2.0,ONSALE,50.0,2019-08-26,2019,8,0.033,-0.016,0.012,0.025


In [40]:
#assign different values to sale and non-sale records
df_total4['GROUP_R'] = np.where(df_total4['GROUP'] == 'ONSALE', -0.10, 0.01)
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,GROUP,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,ONSALE,50.0,2019-08-26,2019,8,0.040,-0.016,0.012,0.025,-0.10
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,NOTONSALE,25.0,2019-08-26,2019,8,0.040,-0.016,0.012,0.025,0.01
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,ONSALE,50.0,2019-08-26,2019,8,0.005,-0.016,0.012,0.025,-0.10
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,NOTONSALE,25.0,2019-08-26,2019,8,0.005,-0.016,0.012,0.025,0.01
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,ONSALE,50.0,2019-08-26,2019,8,0.033,-0.016,0.012,0.025,-0.10


In [41]:
df_total4.month.value_counts(dropna = False).sort_index()

4     37048
5     37048
8     37048
12    37048
Name: month, dtype: int64

In [42]:
#assign values to each month
conditions = [
    (df_total4['month'] == 2),
    (df_total4['month'] ==4),
    (df_total4['month'] ==7)]
choices = [-0.03, 0.02, 0.06]
df_total4['month_R'] = np.select(conditions, choices, default=0.02)


In [43]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,UNITS,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,50.0,2019-08-26,2019,8,0.040,-0.016,0.012,0.025,-0.10,0.02
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,25.0,2019-08-26,2019,8,0.040,-0.016,0.012,0.025,0.01,0.02
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,50.0,2019-08-26,2019,8,0.005,-0.016,0.012,0.025,-0.10,0.02
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,25.0,2019-08-26,2019,8,0.005,-0.016,0.012,0.025,0.01,0.02
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,50.0,2019-08-26,2019,8,0.033,-0.016,0.012,0.025,-0.10,0.02


In [44]:
#generate a random number for each row that is either 1 or -1
import random
df_total4['randNumCol'] = np.random.choice([1, -1], df_total4.shape[0])
df_total4.head(100)

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,date,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019-08-26,2019,8,0.040,-0.016,0.012,0.025,-0.10,0.02,-1
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019-08-26,2019,8,0.040,-0.016,0.012,0.025,0.01,0.02,-1
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019-08-26,2019,8,0.005,-0.016,0.012,0.025,-0.10,0.02,1
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019-08-26,2019,8,0.005,-0.016,0.012,0.025,0.01,0.02,-1
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019-08-26,2019,8,0.033,-0.016,0.012,0.025,-0.10,0.02,-1
5,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019-08-26,2019,8,0.033,-0.016,0.012,0.025,0.01,0.02,1
6,Brandega-17919,Back to School,Botann's,12345-33233-110,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019-08-26,2019,8,0.046,-0.016,0.012,0.025,-0.10,0.02,-1
7,Brandega-17919,Back to School,Botann's,12345-33233-110,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019-08-26,2019,8,0.046,-0.016,0.012,0.025,0.01,0.02,-1
8,Brandega-17919,Back to School,Botann's,12345-28816-111,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019-08-26,2019,8,0.015,-0.016,0.012,0.025,-0.10,0.02,1
9,Brandega-17919,Back to School,Botann's,12345-28816-111,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019-08-26,2019,8,0.015,-0.016,0.012,0.025,0.01,0.02,-1


In [45]:
#sum the generated numbers for each row, making a unique value for each row called total_r

cols = ['Retailer_R','PRODUCT_GROUP_ID_R','Product_Unit_UPC_R','Promo_Name_R','GROUP_R','month_R']
df_total4['TOTAL_R'] = df_total4[cols].astype(float).sum(axis=1)


In [46]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,year,month,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019,8,0.040,-0.016,0.012,0.025,-0.10,0.02,-1,-0.019
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019,8,0.040,-0.016,0.012,0.025,0.01,0.02,-1,0.091
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019,8,0.005,-0.016,0.012,0.025,-0.10,0.02,1,-0.054
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,2019,8,0.005,-0.016,0.012,0.025,0.01,0.02,-1,0.056
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,2019,8,0.033,-0.016,0.012,0.025,-0.10,0.02,-1,-0.026


In [47]:
df_total4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148192 entries, 0 to 148191
Data columns (total 30 columns):
Promo_ID                       148192 non-null object
Promo_Name                     148192 non-null object
Retailer_Name                  148192 non-null object
Retailer_Store_ID              148192 non-null object
Product_Group_ID               148192 non-null object
Product_Unit_UPC               148192 non-null object
Product_Brand                  148192 non-null object
Product_Brand_Category_Desc    148192 non-null object
Product_Unit_Description       148192 non-null object
PRICE                          148192 non-null float64
TPP_Type                       148192 non-null object
TPP_Start                      148192 non-null object
TPP_End                        148192 non-null object
Product_Units_Per_Case         148192 non-null float64
Product_CPG_COGS_Price         148192 non-null float64
SIZE                           148192 non-null object
CASES                 

In [48]:
#create new prices based on the adjustment numbers generated on the previous lines
df_total4['PRICE_R']=df_total4['PRICE']*(1.0-(df_total4['randNumCol']*df_total4['TOTAL_R']))
df_total4['UNITS_R']=df_total4['UNITS']*(1.0+(df_total4['randNumCol']*df_total4['TOTAL_R']))

In [49]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,0.040,-0.016,0.012,0.025,-0.10,0.02,-1,-0.019,7.69104,50.950
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,0.040,-0.016,0.012,0.025,0.01,0.02,-1,0.091,9.61171,22.725
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,0.005,-0.016,0.012,0.025,-0.10,0.02,1,-0.054,8.26336,47.300
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,0.005,-0.016,0.012,0.025,0.01,0.02,-1,0.056,9.30336,23.600
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,0.033,-0.016,0.012,0.025,-0.10,0.02,-1,-0.026,7.63616,51.300


In [50]:
#Make sure that size is always positive, create a new variable called price_size convert size to a numeric value
df_total4.SIZE = df_total4.SIZE.astype(float).fillna(0.0)


df_total4['PRICE_SIZE']=df_total4['PRICE_R']*df_total4['SIZE']


df_total4.SIZE.value_counts(dropna = False).sort_index()

-8.0     74096
 16.0    74096
Name: SIZE, dtype: int64

In [51]:
#create a dummy variable for ice cream
df_total4['SIZE_16'] = np.where(df_total4['SIZE'] == 16, 1, 0)
df_total4['SIZE_16_PRICE']=df_total4['SIZE_16']*df_total4['PRICE']
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,PRICE,...,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R,PRICE_SIZE,SIZE_16,SIZE_16_PRICE
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,0.025,-0.10,0.02,-1,-0.019,7.69104,50.950,123.05664,1,7.84
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,0.025,0.01,0.02,-1,0.091,9.61171,22.725,153.78736,1,8.81
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,0.025,-0.10,0.02,1,-0.054,8.26336,47.300,132.21376,1,7.84
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,8.81,...,0.025,0.01,0.02,-1,0.056,9.30336,23.600,148.85376,1,8.81
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,7.84,...,0.025,-0.10,0.02,-1,-0.026,7.63616,51.300,122.17856,1,7.84


In [52]:
#bring in the stats library to build the model
import statsmodels.api as sm  
#scikit learn is another library that you could use.  Its focus is more on prediction.  This library is more focused on understanding why and how certain variables impact your target.  Which is quantity in this case


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [53]:
df_total4=df_total4.drop(['PRICE'], axis=1)

In [54]:
v = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
w = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
x = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
y = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]
z = df_total4[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE','UNITS_R']]

In [55]:
pricev = pd.DataFrame((v['PRICE_R']*0.95))
pricew = pd.DataFrame((w['PRICE_R']*0.85))
pricey = pd.DataFrame((y['PRICE_R']*0.90))
pricez= pd.DataFrame((z['PRICE_R']*0.80))

qv = pd.DataFrame((v['UNITS_R']*1.05))
qw = pd.DataFrame((w['UNITS_R']*1.15))
qy = pd.DataFrame((y['UNITS_R']*1.10))
qz= pd.DataFrame((z['UNITS_R']*1.20))

In [56]:
df_total4.head()

,Promo_ID,Promo_Name,Retailer_Name,Retailer_Store_ID,Product_Group_ID,Product_Unit_UPC,Product_Brand,Product_Brand_Category_Desc,Product_Unit_Description,TPP_Type,...,Promo_Name_R,GROUP_R,month_R,randNumCol,TOTAL_R,PRICE_R,UNITS_R,PRICE_SIZE,SIZE_16,SIZE_16_PRICE
0,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,TPR,...,0.025,-0.10,0.02,-1,-0.019,7.69104,50.950,123.05664,1,7.84
1,Brandega-17919,Back to School,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,N,...,0.025,0.01,0.02,-1,0.091,9.61171,22.725,153.78736,1,8.81
2,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,TPR,...,0.025,-0.10,0.02,1,-0.054,8.26336,47.300,132.21376,1,7.84
3,Brandega-17919,Back to School,Botann's,12345-79914-108,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,N,...,0.025,0.01,0.02,-1,0.056,9.30336,23.600,148.85376,1,8.81
4,Brandega-17919,Back to School,Botann's,12345-90005-109,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,TPR,...,0.025,-0.10,0.02,-1,-0.026,7.63616,51.300,122.17856,1,7.84


In [57]:
v=v.drop(['PRICE_R','UNITS_R'], axis=1)
v = pd.concat([v, pricev,qv], axis=1)
v['PRICE_POINT']='V'
stuff=df_total4[['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type']]
v = pd.concat([v, stuff], axis=1)

In [58]:

w=w.drop(['PRICE_R','UNITS_R'], axis=1)
w = pd.concat([w, pricew,qw], axis=1)
w['PRICE_POINT']='W'

w = pd.concat([w, stuff], axis=1)

y=y.drop(['PRICE_R','UNITS_R'], axis=1)
y = pd.concat([y, pricey,qy], axis=1)
y['PRICE_POINT']='Y'
y = pd.concat([y, stuff], axis=1)

z=z.drop(['PRICE_R','UNITS_R'], axis=1)
z = pd.concat([z, pricez,qz], axis=1)
z['PRICE_POINT']='Z'
z = pd.concat([z, stuff], axis=1)

x['PRICE_POINT']='X'
x = pd.concat([x, stuff], axis=1)





/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [59]:
xxx=pd.concat([v,w,x,y,z],axis=0)

In [60]:
#build the ols model

independent=xxx[['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE']]
P=xxx['PRICE_POINT']
stuff=xxx[['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type']]

independent = sm.add_constant(independent, prepend=False)

dependent=xxx['UNITS_R']

In [61]:
mod = sm.OLS(dependent, independent)

In [62]:
res = mod.fit()

In [63]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                UNITS_R   R-squared:                       0.733
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                 5.090e+05
Date:                Thu, 17 Jan 2019   Prob (F-statistic):               0.00
Time:                        17:52:22   Log-Likelihood:            -2.8779e+06
No. Observations:              740960   AIC:                         5.756e+06
Df Residuals:                  740955   BIC:                         5.756e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
PRICE_R      -11.5782      0.022   -524.744      0.0

In [64]:
ypred = pd.DataFrame(res.predict(independent))
ypred.columns = ['P_UNITS']

In [65]:
model_output=pd.concat([ypred,independent,stuff,P], axis=1)
model_output.head()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Promo_ID,Retailer_Store_ID,Product_Unit_UPC,TPP_Type,PRICE_POINT
0,47.586930,7.306488,-0.019,16.0,123.05664,1.0,Brandega-17919,12345-94249-107,44309-79016,TPR,V
1,20.394954,9.131125,0.091,16.0,153.78736,1.0,Brandega-17919,12345-94249-107,44309-79016,N,V
2,50.139343,7.850192,-0.054,16.0,132.21376,1.0,Brandega-17919,12345-79914-108,44309-79016,TPR,V
3,27.486428,8.838192,0.056,16.0,148.85376,1.0,Brandega-17919,12345-79914-108,44309-79016,N,V
4,48.970229,7.254352,-0.026,16.0,122.17856,1.0,Brandega-17919,12345-90005-109,44309-79016,TPR,V


In [66]:
df_main_stuff=df_total4.drop(['PRICE_R','TOTAL_R','SIZE','PRICE_SIZE'], axis=1)
df_main_stuff.columns

Index(['Promo_ID', 'Promo_Name', 'Retailer_Name', 'Retailer_Store_ID',
       'Product_Group_ID', 'Product_Unit_UPC', 'Product_Brand',
       'Product_Brand_Category_Desc', 'Product_Unit_Description', 'TPP_Type',
       'TPP_Start', 'TPP_End', 'Product_Units_Per_Case',
       'Product_CPG_COGS_Price', 'CASES', 'GROUP', 'UNITS', 'date', 'year',
       'month', 'Retailer_R', 'PRODUCT_GROUP_ID_R', 'Product_Unit_UPC_R',
       'Promo_Name_R', 'GROUP_R', 'month_R', 'randNumCol', 'UNITS_R',
       'SIZE_16', 'SIZE_16_PRICE'],
      dtype='object')

In [67]:
df_main_stuff=df_main_stuff.drop_duplicates(subset=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type'], keep='first', inplace=False)
df_main_stuff.shape

(148192, 30)

In [68]:
df_total5 =model_output.merge(df_main_stuff, on=['Promo_ID','Retailer_Store_ID','Product_Unit_UPC','TPP_Type'], how='inner')
df_total5.shape

(740960, 37)

In [69]:
df_total5.describe()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Product_Units_Per_Case,Product_CPG_COGS_Price,CASES,UNITS,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
count,740960.000000,740960.000000,740960.000000,740960.000000,740960.000000,740960.0,740960.000000,740960.000000,740960.0,740960.00000,...,740960.000000,740960.000000,740960.000000,740960.000000,740960.000,7.409600e+05,740960.000000,740960.000000,740960.0,740960.000000
mean,53.625097,5.951718,0.024425,4.000000,48.581940,1.0,32.500000,4.840000,1.5,48.75000,...,0.016515,0.000114,0.016545,0.016250,-0.045,2.000000e-02,-0.000999,48.750088,0.5,4.228125
std,19.502251,1.818627,0.074375,12.000008,87.201081,0.0,7.500005,1.350001,0.5,20.11686,...,0.026875,0.015898,0.027906,0.020945,0.055,3.112547e-13,1.000000,20.424695,0.5,4.236198
min,-7.187041,3.082464,-0.170000,-8.000000,-47.712000,1.0,25.000000,3.490000,1.0,25.00000,...,-0.030000,-0.016000,-0.030000,-0.019000,-0.100,2.000000e-02,-1.000000,20.075000,0.0,0.000000
25%,39.492087,4.255020,-0.032000,-8.000000,-37.692480,1.0,25.000000,3.490000,1.0,36.25000,...,-0.008000,-0.015250,-0.012750,0.012500,-0.100,2.000000e-02,-1.000000,31.481250,0.0,0.000000
50%,54.051833,5.775140,0.025000,4.000000,38.969280,1.0,32.500000,4.840000,1.5,45.00000,...,0.018000,-0.002000,0.022000,0.024000,-0.045,2.000000e-02,-1.000000,45.720000,0.5,3.920000
75%,67.620680,7.547085,0.080000,16.000000,132.502400,1.0,40.000000,6.190000,2.0,57.50000,...,0.041000,0.013000,0.041500,0.027750,0.010,2.000000e-02,1.000000,60.535000,1.0,8.345000
max,112.159042,10.545570,0.200000,16.000000,168.729120,1.0,40.000000,6.190000,2.0,80.00000,...,0.059000,0.032000,0.059000,0.036000,0.010,2.000000e-02,1.000000,93.280000,1.0,8.810000


In [70]:
df_total5.loc[df_total5['P_UNITS'] < 0, 'P_UNITS'] = 0
df_total5.describe()

,P_UNITS,PRICE_R,TOTAL_R,SIZE,PRICE_SIZE,const,Product_Units_Per_Case,Product_CPG_COGS_Price,CASES,UNITS,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
count,740960.000000,740960.000000,740960.000000,740960.000000,740960.000000,740960.0,740960.000000,740960.000000,740960.0,740960.00000,...,740960.000000,740960.000000,740960.000000,740960.000000,740960.000,7.409600e+05,740960.000000,740960.000000,740960.0,740960.000000
mean,53.626984,5.951718,0.024425,4.000000,48.581940,1.0,32.500000,4.840000,1.5,48.75000,...,0.016515,0.000114,0.016545,0.016250,-0.045,2.000000e-02,-0.000999,48.750088,0.5,4.228125
std,19.496905,1.818627,0.074375,12.000008,87.201081,0.0,7.500005,1.350001,0.5,20.11686,...,0.026875,0.015898,0.027906,0.020945,0.055,3.112547e-13,1.000000,20.424695,0.5,4.236198
min,0.000000,3.082464,-0.170000,-8.000000,-47.712000,1.0,25.000000,3.490000,1.0,25.00000,...,-0.030000,-0.016000,-0.030000,-0.019000,-0.100,2.000000e-02,-1.000000,20.075000,0.0,0.000000
25%,39.492087,4.255020,-0.032000,-8.000000,-37.692480,1.0,25.000000,3.490000,1.0,36.25000,...,-0.008000,-0.015250,-0.012750,0.012500,-0.100,2.000000e-02,-1.000000,31.481250,0.0,0.000000
50%,54.051833,5.775140,0.025000,4.000000,38.969280,1.0,32.500000,4.840000,1.5,45.00000,...,0.018000,-0.002000,0.022000,0.024000,-0.045,2.000000e-02,-1.000000,45.720000,0.5,3.920000
75%,67.620680,7.547085,0.080000,16.000000,132.502400,1.0,40.000000,6.190000,2.0,57.50000,...,0.041000,0.013000,0.041500,0.027750,0.010,2.000000e-02,1.000000,60.535000,1.0,8.345000
max,112.159042,10.545570,0.200000,16.000000,168.729120,1.0,40.000000,6.190000,2.0,80.00000,...,0.059000,0.032000,0.059000,0.036000,0.010,2.000000e-02,1.000000,93.280000,1.0,8.810000


In [71]:
df_total5=df_total5.rename(index=str, columns={"PRICE_R": "PRICE"})

#tips = df_total5.sort_values(['Promo_ID', 'Retailer_Store_ID', 'Product_Unit_UPC'])
df_total5.head()

,P_UNITS,PRICE,TOTAL_R,SIZE,PRICE_SIZE,const,Promo_ID,Retailer_Store_ID,Product_Unit_UPC,TPP_Type,...,Retailer_R,PRODUCT_GROUP_ID_R,Product_Unit_UPC_R,Promo_Name_R,GROUP_R,month_R,randNumCol,UNITS_R,SIZE_16,SIZE_16_PRICE
0,47.586930,7.306488,-0.019,16.0,123.05664,1.0,Brandega-17919,12345-94249-107,44309-79016,TPR,...,0.04,-0.016,0.012,0.025,-0.1,0.02,-1,50.95,1,7.84
1,56.491806,6.537384,-0.019,16.0,123.05664,1.0,Brandega-17919,12345-94249-107,44309-79016,TPR,...,0.04,-0.016,0.012,0.025,-0.1,0.02,-1,50.95,1,7.84
2,43.134492,7.691040,-0.019,16.0,123.05664,1.0,Brandega-17919,12345-94249-107,44309-79016,TPR,...,0.04,-0.016,0.012,0.025,-0.1,0.02,-1,50.95,1,7.84
3,52.039368,6.921936,-0.019,16.0,123.05664,1.0,Brandega-17919,12345-94249-107,44309-79016,TPR,...,0.04,-0.016,0.012,0.025,-0.1,0.02,-1,50.95,1,7.84
4,60.944244,6.152832,-0.019,16.0,123.05664,1.0,Brandega-17919,12345-94249-107,44309-79016,TPR,...,0.04,-0.016,0.012,0.025,-0.1,0.02,-1,50.95,1,7.84


In [72]:
#keep the relevant variables
df_total5 = df_total5[['Promo_ID',
 'Promo_Name',
 'PRICE_POINT',
'Retailer_Name',
 'Retailer_Store_ID',
 'Product_Group_ID',
 'Product_Unit_UPC',
 'Product_Brand',
 'Product_Brand_Category_Desc',
 'Product_Unit_Description',
 'PRICE',
 'TPP_Type',
 'TPP_Start',
 'TPP_End',
 'Product_Units_Per_Case',
 'Product_CPG_COGS_Price',
 'SIZE',
 'CASES',
 'GROUP',
 'UNITS',     
 'P_UNITS']]

In [73]:
#capitalize the field names to load in dash
df_total5=df_total5.rename(str.upper, axis='columns')

In [74]:
df_total5.PRICE_POINT.value_counts(dropna = False).sort_index()

V    148192
W    148192
X    148192
Y    148192
Z    148192
Name: PRICE_POINT, dtype: int64

In [75]:
#De-Dup the file
df_total5=df_total5.drop_duplicates(subset=['PROMO_ID','RETAILER_STORE_ID','PRODUCT_UNIT_UPC','GROUP','PRICE_POINT'], keep='first', inplace=False)
df_total5.shape
df_total5.head()


,PROMO_ID,PROMO_NAME,PRICE_POINT,RETAILER_NAME,RETAILER_STORE_ID,PRODUCT_GROUP_ID,PRODUCT_UNIT_UPC,PRODUCT_BRAND,PRODUCT_BRAND_CATEGORY_DESC,PRODUCT_UNIT_DESCRIPTION,...,TPP_TYPE,TPP_START,TPP_END,PRODUCT_UNITS_PER_CASE,PRODUCT_CPG_COGS_PRICE,SIZE,CASES,GROUP,UNITS,P_UNITS
0,Brandega-17919,Back to School,V,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,...,TPR,2019-08-26,2019-09-04,25.0,6.19,16.0,2.0,ONSALE,50.0,47.586930
1,Brandega-17919,Back to School,W,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,...,TPR,2019-08-26,2019-09-04,25.0,6.19,16.0,2.0,ONSALE,50.0,56.491806
2,Brandega-17919,Back to School,X,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,...,TPR,2019-08-26,2019-09-04,25.0,6.19,16.0,2.0,ONSALE,50.0,43.134492
3,Brandega-17919,Back to School,Y,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,...,TPR,2019-08-26,2019-09-04,25.0,6.19,16.0,2.0,ONSALE,50.0,52.039368
4,Brandega-17919,Back to School,Z,Botann's,12345-94249-107,MM-SF-TM-B-16,44309-79016,Mitan Munchies,Mitan Munchies Trail Mix Blend 16 oz,Mitan Munchies Chex Mix 16 oz,...,TPR,2019-08-26,2019-09-04,25.0,6.19,16.0,2.0,ONSALE,50.0,60.944244


In [76]:
# The code was removed by Watson Studio for sharing.

In [77]:
df_load=df_total5

In [78]:
#load records to dashdb

import ibmdbpy
from ibmdbpy import IdaDataBase
idadb = idadb = IdaDataBase(dsn="DASHDB;Database=BLUDB;Hostname=" + credentials_1["host"] + ";PROTOCOL=TCPIP;UID=" + credentials_1["username"] + ";PWD=" + credentials_1["password"])

In [79]:
 idadb.as_idadataframe(df_load, "TRAIL_MIX_FINAL_PRODUCTION",clear_existing = True)

DataFrame will be splitted into 1950 chunks. (380 rows per chunk)
Uploaded: 1950/1950... [DONE]


In [80]:
#spark_df = sqlContext.createDataFrame(df_total5)

In [81]:
#### this inserts records


#from ingest.Connectors import Connectors

#dashdbsaveoption = {
 #                    Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
  #                   Connectors.DASHDB.DATABASE          : 'BLUDB',
   #                  Connectors.DASHDB.USERNAME          : 'dash10112',
    #                 Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
     #                Connectors.DASHDB.TARGET_TABLE_NAME : 'ICE_CREAM',
      #               Connectors.DASHDB.TARGET_WRITE_MODE : 'merge' }

#spark_df = spark_df.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

In [82]:
#### this deletes then inserts records


#from ingest.Connectors import Connectors

#dashdbsaveoption = {
 #                    Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
  #                   Connectors.DASHDB.DATABASE          : 'BLUDB',
   #                  Connectors.DASHDB.USERNAME          : 'dash10112',
    #                 Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
     #                Connectors.DASHDB.TARGET_TABLE_NAME : 'DASH10112.ICE_CREAM',
      #               Connectors.DASHDB.TARGET_TABLE_ACTION : 'replace' }

#spark_df = spark_df.write.format("com.ibm.spark.discover").options(**dashdbsaveoption).save()

In [83]:
#This reads records


#from ingest.Connectors import Connectors
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

#dashDBloadOptions = { Connectors.DASHDB.HOST              : 'dashdb-entry-yp-dal09-08.services.dal.bluemix.net',
 #                     Connectors.DASHDB.DATABASE          : 'BLUDB',
  #                    Connectors.DASHDB.USERNAME          : 'dash10112',
   #                   Connectors.DASHDB.PASSWORD          : '-rMyl8IIE1~n',
    #                  Connectors.DASHDB.SOURCE_TABLE_NAME : 'ICE_CREAM'}

#dashdbDF = sqlContext.read.format("com.ibm.spark.discover").options(**dashDBloadOptions).load()
#dashdbDF.printSchema()
#dashdbDF.show()


In [84]:
#abcd=dashdbDF.toPandas() 

In [85]:
#abcd.shape
#abcd.info()